In [2]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [3]:
with sqlite3.connect("../laliga.sqlite") as con:
    df=pd.read_sql_query("SELECT * FROM Matches",con)
    dr=pd.read_sql_query("SELECT * FROM Predictions",con)

df = df.dropna(subset=['score'])

In [4]:
df["score_home_team"] = df["score"].str.split(":").str[0].astype(float)
df["score_away_team"] = df["score"].str.split(":").str[1].astype(float)
df["goal_difference"] = df["score_home_team"] - df["score_away_team"]

df["match_result"] = np.where(df['score_home_team'] > df['score_away_team'], '1', np.where(df['score_home_team'] < df['score_away_team'], '2', 'X'))

In [34]:
def func_home_wins(data):
    return (data[data == '1']).count()

def func_away_wins(data):
    return (data[data == '2']).count()

def func_tie(data):
    return (data[data == 'X']).count()

df_class_home = df.groupby(['division', 'season', 'home_team']).agg(
    GF=pd.NamedAgg(column='score_home_team', aggfunc='sum'),
    GA=pd.NamedAgg(column='score_away_team', aggfunc='sum'),
    W=pd.NamedAgg(column='match_result', aggfunc=func_home_wins),
    L=pd.NamedAgg(column='match_result', aggfunc=func_away_wins),
    T=pd.NamedAgg(column='match_result', aggfunc=func_tie)
).reset_index()

df_class_away = df.groupby(['division', 'season', 'away_team']).agg(
    GF=pd.NamedAgg(column='score_away_team', aggfunc='sum'),
    GA=pd.NamedAgg(column='score_home_team', aggfunc='sum'),
    W=pd.NamedAgg(column='match_result', aggfunc=func_away_wins),
    L=pd.NamedAgg(column='match_result', aggfunc=func_home_wins),
    T=pd.NamedAgg(column='match_result', aggfunc=func_tie)
).reset_index()

df_class_away.rename(columns={'away_team': 'team'}, inplace=True)
df_class_home.rename(columns={'home_team': 'team'}, inplace=True)

df_classification = df_class_away.merge(df_class_home, how='outer')
df_classification = df_classification.groupby(['season', 'team', 'division']).sum().reset_index()
# df_classification['GF'] = df_classification['GF'].shift(1)

df_classification['GD'] = df_classification['GF'] - df_classification['GA']
df_classification['Pts'] = (df_classification['W']) * 3 + df_classification['T']

df_classification['year_of_start'] = df_classification['season'].str.split("-").str[0].astype(int)

df_classification_ordered = df_classification.sort_values(by=['year_of_start'], ascending=False)
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division'], ascending=[False, True])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts'], ascending=[False, True, False])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts', 'GD'], ascending=[False, True, False, False])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts', 'GD', 'GF'], ascending=[False, True, False, False, False])

df_classification_ordered = df_classification_ordered.reset_index(drop=True)
df_classification_ordered['rank'] = df_classification_ordered.groupby(['year_of_start', 'division']).cumcount() + 1

# lowest_rank_div1= df_classification_ordered.groupby(['season'])['rank'].max().reset_index()
# df_classification_ordered_next=df_classification_ordered.merge(lowest_rank_div1,how='outer')

df_classification_1_div = df_classification_ordered[df_classification_ordered['division']==1]

df_classification_2_div = df_classification_ordered[df_classification_ordered['division']==2]
num_teams_1_div = df_classification_1_div.groupby('season')['rank'].max().reset_index()

merged_df = df_classification_2_div.merge(num_teams_1_div, on='season')

merged_df['rank'] = merged_df['rank_x'] + merged_df['rank_y']

merged_df = merged_df.drop(columns={'rank_x','rank_y'})

df_classification_ordered_next = merged_df.merge(df_classification_1_div,how='outer')


df_classification_ordered_next = df_classification_ordered_next.sort_values(by=['year_of_start', 'division', 'Pts', 'GD', 'GF'], ascending=[False, True, False, False, False])


df_classification_ordered_next['delayed_rank'] = df_classification_ordered_next.groupby(['team'])['rank'].shift(-1)

df_with_rank = df_classification_ordered_next[['season','division','team','delayed_rank']]

In [38]:
df_useful = df[['season','division','home_team','match_result','away_team']]

home_team_rank = df_useful.merge(df_with_rank, left_on=['season','division', 'home_team'], right_on=['season','division', 'team'], how='left')
home_team_rank.rename(columns={'delayed_rank': 'home_team_rank'}, inplace=True)
home_team_rank.drop(columns=['team'], inplace=True)

away_team_rank = df_useful.merge(df_with_rank, left_on=['season', 'division', 'away_team'], right_on=['season','division', 'team'], how='left')
away_team_rank.rename(columns={'delayed_rank': 'away_team_rank'}, inplace=True)
away_team_rank.drop(columns=['team'], inplace=True)

df_new = away_team_rank.merge(home_team_rank, on=['season', 'division', 'home_team','away_team'], how='left')

df_new.rename(columns={'match_result_x': 'match_result'},inplace=True)

df_to_train = df_new[['season','home_team','away_team','away_team_rank','home_team_rank','match_result']]
#df_to_train = df_to_train.fillna(0)
df_to_train[df_to_train['season']=='2020-2021'].head(30)

,season,home_team,away_team,away_team_rank,home_team_rank,match_result
25626,2020-2021,SD Eibar,Celta de Vigo,17.0,14.0,X
25627,2020-2021,Granada CF,Athletic,11.0,7.0,1
25628,2020-2021,Cádiz CF,CA Osasuna,10.0,22.0,2
25629,2020-2021,Alavés,Real Betis,15.0,16.0,2
25630,2020-2021,Real Valladolid,Real Sociedad,6.0,13.0,X
25631,2020-2021,Villarreal,SD Huesca,21.0,5.0,X
25632,2020-2021,Valencia,Levante,12.0,9.0,1
25633,2020-2021,Atlético Madrid,Sevilla FC,4.0,3.0,1
25634,2020-2021,Real Madrid,Getafe,8.0,1.0,1
25635,2020-2021,Barcelona,Elche CF,26.0,2.0,1
